In [42]:
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [2]:
train_df = pd.read_csv("../data/raw/train.csv")

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from src.features.preprocess import preprocess
from src.features.vectorizer import get_count_vectorizer

tokenize_fn = word_tokenize

train_tokens = [tokenize_fn(preprocess(text)) for text in train_df["text"]]
embedding_fn = get_count_vectorizer(train_tokens)

KeyboardInterrupt: 

In [42]:
from src.data.dataset import TextDataset

ds = TextDataset("../data/raw/train.csv",
                 preprocess_fn=preprocess,
                 tokenize_fn=tokenize_fn,
                 vectorizer_fn=embedding_fn, sparse=True)

In [43]:
val_ratio = 0.1
num_val = int(val_ratio * len(ds))
num_train = len(ds) - num_val
train_ds, val_ds = torch.utils.data.random_split(ds, [num_train, num_val])
len(train_ds), len(val_ds)

(22259, 2473)

In [38]:
from sklearn.naive_bayes import MultinomialNB
from tqdm.auto import tqdm
clf = MultinomialNB()
train_loader = DataLoader(train_ds, batch_size=1024, shuffle=False)
for batch in tqdm(train_loader):
    X_batch, y_batch = batch
    clf.partial_fit(X_batch.to_dense().numpy(), y_batch.numpy(), classes=[0, 1, 2])
    # break

  0%|          | 0/22 [00:00<?, ?it/s]

In [39]:
val_loader = DataLoader(val_ds, batch_size=512, shuffle=False)
corrects = []
for batch in tqdm(val_loader):
    X_batch, y_batch = batch
    y_batch = y_batch.numpy()
    y_pred = clf.predict(X_batch.to_dense().numpy())
    corrects.append(y_pred == y_batch)
corrects= np.concatenate(corrects)
corrects.sum() / len(corrects)

  0%|          | 0/5 [00:00<?, ?it/s]

0.6332389809947432

In [166]:
# Retrain on full dataset
clf = MultinomialNB()
train_loader = DataLoader(ds, batch_size=512, shuffle=False)
for batch in tqdm(train_loader):
    X_batch, y_batch = batch
    clf.partial_fit(X_batch.to_dense().numpy(), y_batch.numpy(), classes=[0, 1, 2])
    # break

  0%|          | 0/49 [00:00<?, ?it/s]

In [170]:
# Predict on test
test_ds = TextDataset("../data/raw/test.csv",
                 preprocess_fn=preprocess,
                 tokenize_fn=tokenize_fn,
                 embedding_fn=embedding_fn, sparse=True)
test_loader = DataLoader(test_ds, batch_size=512, shuffle=False)

preds = []
for batch in tqdm(test_loader):
    X_batch = batch
    # y_batch = y_batch.numpy()
    y_pred = clf.predict(X_batch.to_dense().numpy())
    preds.append(y_pred)
preds = np.concatenate(preds)

  0%|          | 0/6 [00:00<?, ?it/s]

In [177]:
target_conversion = {
    0: 0,
    1: 1,
    2: -1
}
submission_df = pd.DataFrame({"textID": test_ds.df["textID"], "sentiment": preds})
submission_df["sentiment"] = submission_df['sentiment'].map(target_conversion)
submission_df

,textID,sentiment
0,102f98e5e2,1
1,033b399113,0
2,c125e29be2,1
3,b91e2b0679,-1
4,1a46141274,0
...,...,...
2743,0bfb1006b9,-1
2744,f1df499466,-1
2745,de4da367a4,-1
2746,07a8ec4593,1


In [179]:
submission_df.to_csv("submission.csv", index=False)

In [180]:
!kaggle competitions submit -c eurecom-aml-2022-challenge-3 -f submission.csv -m "Reimplement baseline"

100%|██████████████████████████████████████| 35.5k/35.5k [00:00<00:00, 51.2kB/s]
Successfully submitted to EURECOM AML 2022:: Challenge 3